In [1]:
# Import the required libraries and dependencies
import requests
import pandas as pd
import talib
import numpy as np

from pathlib import Path
from itertools import compress
from matplotlib import pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

#### Download BTC data and create BTC dataframe.

In [2]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    Path('../Resources/BTCUSD_1h.csv'),
    index_col="Date")
df.tail()

,Open,Close,High,Low,Volume
Date,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020


# Identify Candlesticks Pattern
* Based on the TA-Lib module created by Thomas Bulkowsi’s
* Source Code: https://github.com/CanerIrfanoglu/medium/blob/master/candle_stick_recognition/identify_candlestick.py

In [3]:
# extract OHLC 
op = df['Open'].astype(float)
hi = df['High'].astype(float)
lo = df['Low'].astype(float)
cl = df['Close'].astype(float) 


candle_names = talib.get_function_groups()['Pattern Recognition']
engulfing_candle = talib.CDLENGULFING(op, hi, lo, cl)
# create columns for each candle
for candle in engulfing_candle:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = engulfing_candle #getattr(talib, candle)(op, hi, lo, cl)
    
df.head()

,Open,Close,High,Low,Volume,0,100,-100
Date,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0


In [4]:
df['entry'] = np.where(df[0]>0,1,0)
df.head()

,Open,Close,High,Low,Volume,0,100,-100,entry
Date,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0,0


In [5]:
df.head()

,Open,Close,High,Low,Volume,0,100,-100,entry
Date,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0,0


In [6]:
df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'entry']]
df.head()

,Open,Close,High,Low,Volume,entry
Date,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0


In [11]:
df['1h%'] = df['Close'].pct_change()
df['vol_pct_chg'] = df['Volume'].pct_change()

#Calculate TR and ATR
df['prev_close'] = df['Close'].shift(1)
df['TR'] = np.maximum((df['High'] - df['Low']),
                np.maximum(abs(df['High'] - df['prev_close']),
                abs(df['prev_close'] - df['Low'])))

period = 3
df['ATR3'] = df['TR'].rolling(period).sum()/period

df = df.dropna()
df.tail()

,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,prev_close,TR,ATR7
Date,,,,,,,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475,0,-0.005077,0.422752,60514.231682,432.0,780.176404
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709,0,-0.008011,0.728635,60207.000000,664.0,764.176404
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504,0,-0.000268,-0.392852,59724.666301,376.0,763.462119
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219,0,0.011881,-0.204293,59708.630836,989.0,792.176404
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020,0,-0.005776,-0.725653,60418.000000,408.0,528.604976


In [ ]:
# # Assign value 0,1,-1 for No Pattern, Bull Pattern, and Bear Pattern
# df['candlestick_pattern'] = np.nan
# df['candlestick_index'] = np.nan
# for index, row in df.iterrows():

#     # bull pattern 100 or 200
#     if any(row[candle_names].values > 0):
#         pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
#         df.loc[index, 'candlestick_pattern'] = pattern
#         df.loc[index, 'candlestick_index'] = 1
#     # bear pattern -100 or -200
#     elif any(row[candle_names].values < 0):
#         pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
#         df.loc[index, 'candlestick_pattern'] = pattern
#         df.loc[index, 'candlestick_index'] = -1
#     # no pattern
#     else:
#         df.loc[index, 'candlestick_pattern'] = "NO PATTERN"
#         df.loc[index, 'candlestick_index'] = 0

# df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'candlestick_pattern', 'candlestick_index']]
# df['1h%'] = df['Close'].pct_change().dropna()
# df

### Integrating Technical Indicators
#### Simple Moving Average (SMA)

In [13]:
from ta.trend import SMAIndicator

# Add Simple Moving Average (SMA) indicators
df["sma7h"] = SMAIndicator(close=df["Close"], window=7, fillna=True).sma_indicator()
df["sma9h"] = SMAIndicator(close=df["Close"], window=9, fillna=True).sma_indicator()
df["sma20h"] = SMAIndicator(close=df["Close"], window=20, fillna=True).sma_indicator()
df["sma200d"] = SMAIndicator(close=df["Close"], window=4800, fillna=True).sma_indicator()

# Review the DataFrame
df.tail()

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,prev_close,TR,ATR7,sma7h,sma9h,sma20h,sma200d
Date,,,,,,,,,,,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475,0,-0.005077,0.422752,60514.231682,432.0,780.176404,60046.752999,59953.696777,59872.711649,45962.100038
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709,0,-0.008011,0.728635,60207.000000,664.0,764.176404,60047.562470,59965.437477,59863.354399,45962.582677
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504,0,-0.000268,-0.392852,59724.666301,376.0,763.462119,60060.509733,59973.396459,59837.312487,45963.079267
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219,0,0.011881,-0.204293,59708.630836,989.0,792.176404,60270.509733,60051.063126,59872.362487,45963.630517
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020,0,-0.005776,-0.725653,60418.000000,408.0,528.604976,60205.366876,60101.174237,59854.290011,45964.121463


#### Bollinger Bands

In [14]:
from ta.volatility import BollingerBands

# Add Bollinger Bands indicator
indicator_bb = BollingerBands(close=df["Close"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

# Review the DataFrame
df.head(20)

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,prev_close,TR,ATR7,sma7h,sma9h,sma20h,sma200d,bb_bbm,bb_bbh,bb_bbl
Date,,,,,,,,,,,,,,,,,,
2017-01-02 12:00:00,1034.4,1030.0,1035.0,1023.8,1015.054405,0,-0.004350,-0.200758,1034.5,11.2,7.814286,1030.000000,1030.000000,1030.000000,1030.000000,NaN,NaN,NaN
2017-01-02 13:00:00,1029.7,1018.9,1030.0,1005.5,2007.570108,0,-0.010777,0.977796,1030.0,24.5,10.385714,1024.450000,1024.450000,1024.450000,1024.450000,NaN,NaN,NaN
2017-01-02 14:00:00,1020.8,1028.2,1030.0,1018.1,1208.616547,0,0.009127,-0.397970,1018.9,11.9,11.400000,1025.700000,1025.700000,1025.700000,1025.700000,NaN,NaN,NaN
2017-01-02 15:00:00,1028.1,1030.0,1030.2,1027.2,651.648150,0,0.001751,-0.460831,1028.2,3.0,10.642857,1026.775000,1026.775000,1026.775000,1026.775000,NaN,NaN,NaN
2017-01-02 16:00:00,1030.0,1028.4,1033.2,1024.5,1319.624926,0,-0.001553,1.025057,1030.0,8.7,10.771429,1027.100000,1027.100000,1027.100000,1027.100000,NaN,NaN,NaN
2017-01-02 17:00:00,1028.2,1026.7,1029.7,1021.2,596.130692,0,-0.001653,-0.548257,1028.4,8.5,11.128571,1027.033333,1027.033333,1027.033333,1027.033333,NaN,NaN,NaN
2017-01-02 18:00:00,1026.7,1020.2,1029.7,1018.8,960.173306,0,-0.006331,0.610676,1026.7,10.9,11.242857,1026.057143,1026.057143,1026.057143,1026.057143,NaN,NaN,NaN
2017-01-02 19:00:00,1020.2,1012.5,1021.8,1010.0,974.794381,0,-0.007548,0.015228,1020.2,11.8,11.328571,1023.557143,1024.362500,1024.362500,1024.362500,NaN,NaN,NaN
2017-01-02 20:00:00,1012.5,1023.7,1024.7,994.1,3992.610184,0,0.011062,3.095849,1012.5,30.6,12.200000,1024.242857,1024.288889,1024.288889,1024.288889,NaN,NaN,NaN


In [15]:
df.dropna(inplace=True)
df.head()

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,prev_close,TR,ATR7,sma7h,sma9h,sma20h,sma200d,bb_bbm,bb_bbh,bb_bbl
Date,,,,,,,,,,,,,,,,,,
2017-01-03 07:00:00,1020.9,1021.2,1023.8,1016.0,610.489164,0,0.000098,0.621427,1021.1,7.8,7.000000,1024.600000,1024.044444,1023.550,1023.550000,1023.550,1032.833857,1014.266143
2017-01-03 08:00:00,1021.3,1018.1,1021.5,1016.3,435.108331,0,-0.003036,-0.287279,1021.2,5.2,6.928571,1023.385714,1023.911111,1022.955,1023.290476,1022.955,1032.032109,1013.877891
2017-01-03 09:00:00,1018.4,1016.0,1022.1,1013.5,697.653988,0,-0.002063,0.603403,1018.1,8.6,6.428571,1022.114286,1022.922222,1022.810,1022.959091,1022.810,1032.227834,1013.392166
2017-01-03 10:00:00,1016.2,1019.1,1021.4,1013.5,428.319516,0,0.003051,-0.386057,1016.0,7.9,6.685714,1020.857143,1022.088889,1022.355,1022.791304,1022.355,1031.564229,1013.145771
2017-01-03 11:00:00,1019.7,1022.4,1022.7,1019.2,197.723970,0,0.003238,-0.538373,1019.1,3.6,6.471429,1020.428571,1021.811111,1021.975,1022.775000,1021.975,1030.492247,1013.457753


In [16]:
df.to_csv(Path('../../../LSTM testing/test.csv'))